In [1]:
from db.data import Data
import gurobipy as gp
from gurobipy import GRB

import pandas as pd

In [2]:
data = Data()

demand = data.get_demand()
waiting = data.get_waiting()
distances = data.get_distances()
crime = data.get_crime()

In [3]:
RADIUS = 2000
MAX_OFFICERS = 9

In [4]:
distances['in_range'] = (distances['distance'] <= RADIUS).astype(int)

In [5]:
distances['in_range'] = (distances['distance'] <= RADIUS).astype(int)
distances = distances.pivot(index='WPname',columns='DAuid',values='in_range')

In [6]:
distances = distances.loc[:, (distances.sum() > 0)]
distances.head()

DAuid,59151910,59151913,59151914,59151915,59151918,59151919,59151920,59151921,59151922,59151923,...,59153375,59153386,59153388,59153455,59153535,59153563,59153719,59153720,59153919,59153920
WPname,,,,,,,,,,,,,,,,,,,,,
APH Matthew Park,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
Aria Banquet Convention Center,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Bear Creek Park,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,1,1
Big plaza at 88 and scott road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bolivar Park,1,1,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
walleyDA = list(distances.columns)
demand = demand[demand['DAuid'].isin(walleyDA)]

In [8]:
coverage = {}
for index, row in distances.iterrows():
    coverage[index] = [set(row[row==1].keys())]

In [9]:
pop_dict = pd.Series(demand.population_val.values,index=demand.DAuid).to_dict()

In [10]:
demand

,DAuid,population_val,DAarea,lat,lon,FEDcode,FEDName
56,59151910,555.0,0.2230,49.211293,-122.830840,32,Surrey-Centre/Whalley
59,59151913,462.0,2.3874,49.212635,-122.845019,32,Surrey-Centre/Whalley
60,59151914,528.0,0.2372,49.210042,-122.840417,32,Surrey-Centre/Whalley
61,59151915,639.0,0.2614,49.207381,-122.839230,32,Surrey-Centre/Whalley
64,59151918,591.0,0.2419,49.200283,-122.840601,32,Surrey-Centre/Whalley
...,...,...,...,...,...,...,...
545,59153563,473.0,0.0559,49.186924,-122.839187,32,Surrey-Centre/Whalley
556,59153719,514.0,0.1000,49.171313,-122.836222,32,Surrey-Centre/Whalley
557,59153720,511.0,0.7199,49.175867,-122.836148,32,Surrey-Centre/Whalley
604,59153919,818.0,0.1796,49.169438,-122.830393,32,Surrey-Centre/Whalley


In [11]:
regions, population = gp.multidict(pop_dict)
waits, covered = gp.multidict(coverage)

In [12]:
m = gp.Model("SPMARP")
build = m.addVars(list(waiting['name']), vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(list(distances.columns), vtype=GRB.BINARY, name="Is_covered")

Restricted license - for non-production use only - expires 2022-01-13


In [13]:
m.addConstrs((gp.quicksum(build[t] for t in waits if r in covered[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")

{59151910: <gurobi.Constr *Awaiting Model Update*>,
 59151913: <gurobi.Constr *Awaiting Model Update*>,
 59151914: <gurobi.Constr *Awaiting Model Update*>,
 59151915: <gurobi.Constr *Awaiting Model Update*>,
 59151918: <gurobi.Constr *Awaiting Model Update*>,
 59151919: <gurobi.Constr *Awaiting Model Update*>,
 59151920: <gurobi.Constr *Awaiting Model Update*>,
 59151921: <gurobi.Constr *Awaiting Model Update*>,
 59151922: <gurobi.Constr *Awaiting Model Update*>,
 59151923: <gurobi.Constr *Awaiting Model Update*>,
 59151924: <gurobi.Constr *Awaiting Model Update*>,
 59151925: <gurobi.Constr *Awaiting Model Update*>,
 59151926: <gurobi.Constr *Awaiting Model Update*>,
 59151927: <gurobi.Constr *Awaiting Model Update*>,
 59151929: <gurobi.Constr *Awaiting Model Update*>,
 59151931: <gurobi.Constr *Awaiting Model Update*>,
 59151932: <gurobi.Constr *Awaiting Model Update*>,
 59151933: <gurobi.Constr *Awaiting Model Update*>,
 59151934: <gurobi.Constr *Awaiting Model Update*>,
 59151942: <

In [14]:
m.addConstr(gp.quicksum(build[t] for t in waits) == MAX_OFFICERS, name="officers")

<gurobi.Constr *Awaiting Model Update*>

In [15]:
m.setObjective(is_covered.prod(population), GRB.MAXIMIZE)

In [16]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 128 rows, 146 columns and 487 nonzeros
Model fingerprint: 0x40c5b735
Variable types: 0 continuous, 146 integer (146 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+02, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 9e+00]
Found heuristic solution: objective 78402.000000
Presolve removed 72 rows and 72 columns
Presolve time: 0.00s
Presolved: 56 rows, 74 columns, 275 nonzeros
Variable types: 0 continuous, 74 integer (74 binary)

Root relaxation: objective 9.890400e+04, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    98904.000000 98904.0000  0.00%     -    0s

Explored 0 nodes (7 simplex iterations) in 0.04 seconds
Th

In [17]:
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        print(f"\n Place officer at location {tower}.")


 Place officer at location Bear Creek Park.

 Place officer at location Big plaza at 88 and scott road.

 Place officer at location LA Matheson Secondary School/Moffat Park.

 Place officer at location Aria Banquet Convention Center.

 Place officer at location Bolivar Park.

 Place officer at location David Brankin Elementary School/William Beagle Park.

 Place officer at location Kwantlen Park.

 Place officer at location King George Skytrain.

 Place officer at location Surrey Central.


In [18]:
total_population = 0

for region in regions:
    total_population += population[region]

percent = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the patrol officer distribution is: {percent} %")


 The population coverage associated to the patrol officer distribution is: 99.28 %
